In [1]:
#!sudo lsb_release -a

In [2]:
!pip install dgl-0.3-cp37-cp37m-manylinux1_x86_64.whl

In [3]:
!pip uninstall -y networkx

Uninstalling networkx-2.5:
  Successfully uninstalled networkx-2.5


In [4]:
!pip install networkx-2.5-py3-none-any.whl

Processing ./networkx-2.5-py3-none-any.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.7 which is incompatible.


In [5]:
!pip uninstall -y imgaug

Uninstalling imgaug-0.2.7:
  Successfully uninstalled imgaug-0.2.7


In [6]:
!pip install imgaug-0.2.7-py3-none-any.whl

Processing ./imgaug-0.2.7-py3-none-any.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.7 which is incompatible.


In [7]:
#!pip install https://s3.us-east-2.amazonaws.com/dgl.ai/wheels/dgl-0.3-cp35-cp35m-win_amd64.whl
#!pip install https://s3.us-east-2.amazonaws.com/dgl.ai/wheels/dgl-0.3-cp36-cp36m-win_amd64.whl
#!pip install https://s3.us-east-2.amazonaws.com/dgl.ai/wheels/dgl-0.3-cp37-cp37m-win_amd64.whl

In [8]:
#!pip install dgl
#install -c dglteam dgl-cuda11.0
#!pip install dgl-cu110
#!pip install tensorboardX

In [9]:
#!pip uninstall tensorboardX

In [10]:
!pip install tensorboardX-2.1-py2.py3-none-any.whl

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os
path="/content/drive/MyDrive/"
#path="/content/"
os.chdir(path)
os.listdir(path)
!python utils.py
!python utils_data.py
!python utils_layers.py

In [13]:

import argparse

import json

import os
import time

import dgl.init

import numpy as np

import tensorboardX
import torch as th
import torch.nn.functional as F
from torch.autograd import Variable
import utils_data
from utils_layers import GeomGCNNet

In [40]:
args={'model':'GeomGCN_TwoLayers',
  'dataset':'cora',
  'dataset_embedding':'poincare',
  'num_hidden':32,
  'num_heads_layer_one':1,
  'num_heads_layer_two':1,
  'layer_one_ggcn_merge':'cat',
  'layer_two_ggcn_merge':'mean',
  'layer_one_channel_merge':'cat',
  'layer_two_channel_merge':'mean',
  'dropout_rate':0.5,
  'learning_rate':0.2,
  'weight_decay_layer_one':5e-06,
  'weight_decay_layer_two':5e-06,
  'num_epochs_patience':100,
  'num_epochs_max':5000,
  'run_id':0,
  'dataset_split':'splits/cora_split_0.6_0.2_0.npz',
  'learning_rate_decay_patience':50,
  'learning_rate_decay_factor':0.8
   }

In [15]:
import os
path="."
#path="/content/"
os.chdir(path)
os.listdir(path)

['文稿',
 '照片',
 '软件安装包',
 '资源种子',
 'Avengers.Infinity.War.2018.2160p.BluRay.REMUX.HEVC.DTS-HD.MA.TrueHD.7.1.Atmos-FGT.mkv',
 'EBSCO',
 'Colab Notebooks',
 'Untitled0.ipynb',
 '未命名项目.gscript',
 'Untitled1.ipynb',
 'Untitled (2)',
 'Lab 4 ex.ipynb',
 'DL_lab2.ipynb',
 'Untitled (1)',
 '__pycache__',
 '.ipynb_checkpoints',
 'data',
 'embedding_method_combinations_all',
 'splits',
 'utils.py',
 'Anyfile Notepad Files',
 'runs',
 'dgl-0.3-cp35-cp35m-manylinux1_x86_64.whl',
 'dgl-0.3-cp36-cp36m-manylinux1_x86_64.whl',
 'dgl-0.3-cp37-cp37m-manylinux1_x86_64.whl',
 'dgl-0.3-cp37-cp37m-win_amd64.whl',
 'dgl-0.4rc191005-cp37-cp37m-manylinux1_x86_64.whl',
 'structural_neighborhood',
 'networkx-2.5-py3-none-any.whl',
 'Untitled',
 'imgaug-0.2.7-py3-none-any.whl',
 'tensorboardX-2.1-py2.py3-none-any.whl',
 'utils_data.py',
 'utils_layers.py',
 'Others',
 'test.csv',
 'test_GeomGCN_TwoLayers_ExperimentTwoAll_cora_poincare.csv',
 'Test_Result',
 'Other_GCN',
 'new_data',
 'unconnected_nodes']

In [41]:
import pandas as pd

if __name__ == '__main__':
    record = pd.DataFrame(columns=['Train Loss', 'Val Loss', 'Train Acc', 'Val Acc'])
    '''
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str)
    parser.add_argument('--num_hidden', type=int)
    parser.add_argument('--num_heads_layer_one', type=int)
    parser.add_argument('--num_heads_layer_two', type=int)
    parser.add_argument('--layer_one_ggcn_merge', type=str, default='cat')
    parser.add_argument('--layer_two_ggcn_merge', type=str, default='mean')
    parser.add_argument('--layer_one_channel_merge', type=str, default='cat')
    parser.add_argument('--layer_two_channel_merge', type=str, default='mean')
    parser.add_argument('--dropout_rate', type=float)
    parser.add_argument('--learning_rate', type=float)
    parser.add_argument('--weight_decay_layer_one', type=float)
    parser.add_argument('--weight_decay_layer_two', type=float)
    parser.add_argument('--num_epochs_patience', type=int, default=100)
    parser.add_argument('--num_epochs_max', type=int, default=5000)
    parser.add_argument('--run_id', type=str)
    parser.add_argument('--dataset_split', type=str)
    parser.add_argument('--learning_rate_decay_patience', type=int, default=50)
    parser.add_argument('--learning_rate_decay_factor', type=float, default=0.8)
    args = parser.parse_args()
    vars(args)['model'] = 'GeomGCN_TwoLayers_ExperimentTwoAll'
    '''

    #print("test")

    t1 = time.time()

    if args['dataset_split'] == 'jknet':
        g, features, labels, train_mask, val_mask, test_mask, num_features, num_labels = utils_data.load_data(
            args.dataset, None, 0.6, 0.2, 'GeomGCN',args['dataset_embedding'])
    else:
        g, features, labels, train_mask, val_mask, test_mask, num_features, num_labels = utils_data.load_data(
            args['dataset'], args['dataset_split'], None, None, 'GeomGCN', args['dataset_embedding'])
        
    print(time.time() - t1)

    # 图初始化
    g.set_n_initializer(dgl.init.zero_initializer)
    g.set_e_initializer(dgl.init.zero_initializer)
    
    #构建网络
    net = GeomGCNNet(g=g, num_input_features=num_features, num_output_classes=num_labels, num_hidden=args["num_hidden"],
                     num_divisions=25, dropout_rate=args["dropout_rate"],
                     num_heads_layer_one=args["num_heads_layer_one"], num_heads_layer_two=args["num_heads_layer_two"],
                     layer_one_ggcn_merge=args["layer_one_ggcn_merge"],
                     layer_one_channel_merge=args["layer_one_channel_merge"],
                     layer_two_ggcn_merge=args["layer_two_ggcn_merge"],
                     layer_two_channel_merge=args["layer_two_channel_merge"])

    #构建优化器
    optimizer = th.optim.Adam([{'params': net.geomgcn1.parameters(), 'weight_decay': args["weight_decay_layer_one"]},
                               {'params': net.geomgcn2.parameters(), 'weight_decay': args["weight_decay_layer_two"]}],
                              lr=args["learning_rate"])
    
    learning_rate_scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,factor=args["learning_rate_decay_factor"],patience=args["learning_rate_decay_patience"])
    writer = tensorboardX.SummaryWriter(logdir=f'runs/{args["model"]}_{args["run_id"]}')

    features = features.to("cpu")
    labels = labels.to("cpu")
    train_mask = train_mask.to("cpu")
    val_mask = val_mask.to("cpu")
    test_mask = test_mask.to("cpu")

    # Adapted from https://github.com/PetarV-/GAT/blob/master/execute_cora.py
    patience = args["num_epochs_patience"]
    vlss_mn = np.inf
    vacc_mx = 0.0
    vacc_early_model = None
    vlss_early_model = None
    state_dict_early_model = None
    curr_step = 0

    print("epoch")
    # Adapted from https://docs.dgl.ai/tutorials/models/1_gnn/1_gcn.html
    dur = []
    test_time = 0.0
    for epoch in range(20):
        t0 = time.time()

        net.train()
        train_logits = net(features)
        train_logp = F.log_softmax(train_logits, 1)
        train_loss = F.nll_loss(train_logp[train_mask], labels[train_mask])
        #train_loss = Variable(train_loss.data, requires_grad=True)

        train_pred = train_logp.argmax(dim=1)
        train_acc = th.eq(train_pred[train_mask], labels[train_mask]).float().mean().item()

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        net.eval()
        with th.no_grad():
            val_logits = net(features)
            val_logp = F.log_softmax(val_logits, 1)
            val_loss = F.nll_loss(val_logp[val_mask], labels[val_mask]).item()
            val_pred = val_logp.argmax(dim=1)
            val_acc = th.eq(val_pred[val_mask], labels[val_mask]).float().mean().item()

        learning_rate_scheduler.step(val_loss)

        dur.append(time.time() - t0)

        print(
            "Epoch {:05d} | Train Loss {:.4f} | Train Acc {:.4f} | Val Loss {:.4f} | Val Acc {:.4f} | Time(s) {:.4f}".format(
                epoch, train_loss.item(), train_acc, val_loss, val_acc, sum(dur) / len(dur)))

        writer.add_scalar('Train Loss', train_loss.item(), epoch)
        writer.add_scalar('Val Loss', val_loss, epoch)
        writer.add_scalar('Train Acc', train_acc, epoch)
        writer.add_scalar('Val Acc', val_acc, epoch)

        test_time += (sum(dur) / len(dur))

        new={'Train Loss':train_loss.item(),'Val Loss':val_loss,'Train Acc':train_acc,'Val Acc':val_acc}
        
        record=record.append(new,ignore_index=True)   # ignore_index=True,表示不按原来的索引，从0开始自动递增

        # Adapted from https://github.com/PetarV-/GAT/blob/master/execute_cora.py
        if val_acc >= vacc_mx or val_loss <= vlss_mn:
            if val_acc >= vacc_mx and val_loss <= vlss_mn:
                vacc_early_model = val_acc
                vlss_early_model = val_loss
                state_dict_early_model = net.state_dict()
            vacc_mx = np.max((val_acc, vacc_mx))
            vlss_mn = np.min((val_loss, vlss_mn))
            curr_step = 0
        else:
            curr_step += 1
            if curr_step >= patience:
                print()
#                break

    #drive.mount("/content")
    record.to_csv('Test_Result/test_{model}_{dataset}_{embedding}_{rec_time}.csv'.format(model=args["model"], dataset=args["dataset"], embedding=args['dataset_embedding'], rec_time=test_time))
    print(test_time)

    net.load_state_dict(state_dict_early_model)
    net.eval()
    with th.no_grad():
        test_logits = net(features)
        test_logp = F.log_softmax(test_logits, 1)
        test_loss = F.nll_loss(test_logp[test_mask], labels[test_mask]).item()
        test_pred = test_logp.argmax(dim=1)
        test_acc = th.eq(test_pred[test_mask], labels[test_mask]).float().mean().item()
        test_hidden_features = net.geomgcn1(features).cpu().numpy()

        final_train_pred = test_pred[train_mask].cpu().numpy()
        final_val_pred = test_pred[val_mask].cpu().numpy()
        final_test_pred = test_pred[test_mask].cpu().numpy()

    '''
    results_dict = vars(args)
    results_dict['test_loss'] = test_loss
    results_dict['test_acc'] = test_acc
    results_dict['actual_epochs'] = 1 + epoch
    results_dict['val_acc_max'] = vacc_mx
    results_dict['val_loss_min'] = vlss_mn
    results_dict['total_time'] = sum(dur)
    with open(os.path.join('runs', f'{args.model}_{args.run_id}_results.txt'), 'w') as outfile:
        outfile.write(json.dumps(results_dict) + '\n')
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_hidden_features.npz'),
                        hidden_features=test_hidden_features)
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_final_train_predictions.npz'),
                        final_train_predictions=final_train_pred)
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_final_val_predictions.npz'),
                        final_val_predictions=final_val_pred)
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_final_test_predictions.npz'),
                        final_test_predictions=final_test_pred)
    '''                   

test1
test2
test3
test6
<class 'dgl.graph.DGLGraph'>


/usr/local/lib/python3.7/dist-packages/dgl/base.py:18: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  warnings.warn(msg)


test_end
5.248006582260132
图初始化
构建网络
构建优化器
设置学习率
开始epoch
Epoch 00000 | Train Loss 1.9459 | Train Acc 0.1267 | Val Loss 1.9446 | Val Acc 0.2965 | Time(s) 1.6282
Epoch 00001 | Train Loss 1.9440 | Train Acc 0.2970 | Val Loss 1.9367 | Val Acc 0.2965 | Time(s) 1.6325
Epoch 00002 | Train Loss 1.9342 | Train Acc 0.2970 | Val Loss 1.9182 | Val Acc 0.2965 | Time(s) 1.6131
Epoch 00003 | Train Loss 1.9112 | Train Acc 0.2970 | Val Loss 1.8911 | Val Acc 0.2977 | Time(s) 1.6061
Epoch 00004 | Train Loss 1.8739 | Train Acc 0.2970 | Val Loss 1.8603 | Val Acc 0.2977 | Time(s) 1.6023
Epoch 00005 | Train Loss 1.8290 | Train Acc 0.2970 | Val Loss 1.8274 | Val Acc 0.2965 | Time(s) 1.5984
Epoch 00006 | Train Loss 1.7813 | Train Acc 0.2970 | Val Loss 1.7924 | Val Acc 0.2965 | Time(s) 1.5958
Epoch 00007 | Train Loss 1.7429 | Train Acc 0.2978 | Val Loss 1.7559 | Val Acc 0.2965 | Time(s) 1.5911
Epoch 00008 | Train Loss 1.7020 | Train Acc 0.2970 | Val Loss 1.7170 | Val Acc 0.2965 | Time(s) 1.5897
Epoch 00009 | Tr

KeyboardInterrupt: ignored